# 1번 신경망

In [1]:
import torch
import torch.nn as nn #신경망 생성
import torch.nn.functional as F

class Net(nn.Module):  # nn.Module은 층과 전방전파(입력->출력)을 쉽게 구현
    def __init__(self):
        super(Net, self).__init__()
        
        # 1 input image channel, 6 ouput channels, 5x5 square convolution
        # kernel  컨볼루션층
        self.conv1 = nn.Conv2d(1,6,5)
        self.conv2 = nn.Conv2d(6,16,5)
        # an affine operation y = Wx+b   FC층(fully-connected)
        self.fc1 = nn.Linear(16*5*5,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
        
    #forward(): autograd를 이용-> 연산 그래프의 gradient를 구하는 backward가 자동적으로 정의
    # Tensor를 이용할 수 있는 다양한 연산들 사용하여 정의 가능 
    def forward(self,x):   
        # Max pooling over a (2,2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)),2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features*=s
        return num_features

(1) 화면 출력확인 및 의미를 서술

In [2]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


(3) 화면 출력 확인

In [3]:
#nn.parameters : 모듈이 지정되면 매개변수들을 자동으로 관리하는 Tensor의 하나.
params=list(net.parameters())
print(len(params))
print(params[0].size())  #conv1's weight

10
torch.Size([6, 1, 5, 5])


(4) 화면 출력확인

In [4]:
#다음의 임의의 32*32 입력을 가정함
#참고로 크기가 다른 입력을 받을때는 입력의 크기를 재조정하거나 신경망 수정함
input=torch.randn(1,1,32,32)
out=net(input)
print(out)

tensor([[-0.0196,  0.0602,  0.0632,  0.0713, -0.0312, -0.0689,  0.0330, -0.1148,
         -0.0247,  0.0026]], grad_fn=<ThAddmmBackward>)


In [5]:
#오류역전파를 통해 gradient를 구하기전에 모든 가중치의 gradient 버퍼들을 초기화
net.zero_grad()
out.backward(torch.randn(1,10))

In [6]:
#손실함수 정의 및 임의의 값들에 대해서 오차 결과 확인
#nn package는 많이 사용되는 손실함수들을 제공, 여기선 MSE 사용
output=net(input)
target=torch.randn(10)  #a dummy target for example
target = target.view(1,-1) #make it the same shape as output
criterion=nn.MSELoss()

(5) 화면 출력확인

In [7]:
loss=criterion(output, target)
print(loss)

tensor(0.9517, grad_fn=<MseLossBackward>)


(6) 화면 출력확인

In [8]:
net.zero_grad()

In [9]:
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])


(7) 화면 출력확인

In [10]:
loss.backward()

In [11]:
print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad after backward
tensor([ 0.0118, -0.0043, -0.0064, -0.0154,  0.0081, -0.0153])


In [12]:
# 스토캐스틱 경사하강법(미래가중치=현재가중치-학습률*gradinent)을 이용하여 가중치 갱신하는 코드
learning_rate=0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [13]:
# 하지만 위 구현 코드보다 실제, torch.optim에서 구현되는 SDG,Adam,RMSProp 등을 사용함
#오류 역전파에서 최적화하는 방법을 보인 예제 코드

import torch.optim as optim

#create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

#inyour training loop
optimizer.zero_grad()  #zero the gradient buffers
output=net(input)
loss= criterion(output,  target)
loss.backward()
optimizer.step()  #Does the update